# Creating DataFrame and Necessary Import Statements

In [38]:
! pip install fancyimpute

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from cycler import cycler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from fancyimpute import IterativeImputer
from xgboost import XGBClassifier

  Using cached fancyimpute-0.7.0-py3-none-any.whl
  Using cached nose-1.3.7-py3-none-any.whl (154 kB)
  Using cached scikit_learn-1.0.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (23.2 MB)
  Using cached knnimpute-0.1.0-py3-none-any.whl
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [4]:
!pip install kaggle --upgrade

### Hide Kaggle Key

In [6]:
!echo "{\"username\":\"tombohorig\",\"key\":\"81f7fcd3a57da42eb3f3c20e0417fdfe\"}" > kaggle.json

In [7]:
!sudo mkdir -p ~/.kaggle
!sudo cp /content/kaggle.json ~/.kaggle/kaggle.json

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
!kaggle --version

Kaggle API 1.5.4


In [10]:
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 53.3MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 35.7MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 53.9MB/s]
 79% 41.0M/52.2M [00:00<00:00, 60.0MB/s]
100% 52.2M/52.2M [00:00<00:00, 76.3MB/s]
 98% 57.0M/58.3M [00:01<00:00, 38.0MB/s]
100% 58.3M/58.3M [00:01<00:00, 46.5MB/s]


In [11]:
np.random.seed(314159)
train_txn = pd.read_csv('/content/train_transaction.csv.zip', compression='zip')
train_id = pd.read_csv('/content/train_identity.csv.zip', compression='zip')

In [12]:
train_txn.shape

(590540, 394)

In [13]:
train_id.shape

(144233, 41)

# Cleaning Data

### Drop columns with null values over a certain threshold

In [14]:
def remove_heavy_null_columns(df, threshold, n_rows):
  cols_over_threshold = []
  for column in df:
    missing_values = df[column].isna().sum()
    if missing_values != 0 and missing_values/n_rows >= threshold:
      cols_over_threshold.append(column)
  df.drop(cols_over_threshold, axis=1, inplace=True)

threshold = 0.5
remove_heavy_null_columns(df=train_txn, threshold=threshold, n_rows=train_txn.shape[0])
remove_heavy_null_columns(df=train_id, threshold=threshold, n_rows=train_txn.shape[0])

### Perform MICE Imputation on remaining Nan Cells
After deciding to use a XGBoost Model and learning that it can deal with Nan Values and learn from them I will not impute these values

In [ ]:
# def fill_missing_values(df):
#   # get all column names:
#   print(df.head(20))
#   column_names = df.columns
#   # mice = IterativeImputer()
#   # return pd.DataFrame(mice.fit_transform(df), columns=df.columns)

# # print("BEFORE IMPUTATION:")
# # print(train_id.isna().sum())
# # # print(penguins.isna().sum())

# fill_missing_values(df=train_id)

# # print()
# # print("AFTER IMPUTATION:")
# # print(train_id.isna().sum())
# # # print(penguins.isna().sum())

# Convert columns with String to be One Hot Encoded as converting to numeric in same column will give undesired order precedence

In [15]:
# One hot encodings:
def one_hot_encoding(df, column_name):
  encoding = pd.get_dummies(df[column_name])
  df = df.drop(column_name, axis = 1)
  df = df.join(encoding)  
  return df

Preform Cleaning on train_id

In [22]:
# Create maps for cols that should be one hot encoded, but I run out of RAM:
def get_mapped_vals(df, feature):
  unique_vals = df[feature].unique()
  map = {}
  for i in range(len(unique_vals) ):
    map[unique_vals[i] ] = i
  return map

In [23]:
# Transormations:
boolean_found = {'NotFound': 0, 'Found': 1, None: 0}
boolean_new = {'New': 0, 'Found': 1, None: 0}
true_false = {'T': 2, 'F': 1, None: 0}
device_type = {None: 0, 'mobile': 1, 'desktop': 2}
train_id['id_12'].replace(boolean_found, inplace=True)
train_id['id_16'].replace(boolean_found, inplace=True)
train_id['id_27'].replace(boolean_found, inplace=True)
train_id['id_28'].replace(boolean_new, inplace=True)
train_id['id_29'].replace(boolean_found, inplace=True)
train_id['id_35'].replace(true_false, inplace=True)
train_id['id_36'].replace(true_false, inplace=True)
train_id['id_37'].replace(true_false, inplace=True)
train_id['id_38'].replace(true_false, inplace=True)
train_id['DeviceType'].replace(device_type, inplace=True)

# One Hot Encodings:
train_id = one_hot_encoding(df=train_id, column_name='id_15')
train_id = one_hot_encoding(df=train_id, column_name='id_23')
train_id = one_hot_encoding(df=train_id, column_name='id_34')

# I must drop transfrom instead of one-hot encode b/c I run out of RAM:
train_id['id_30'].replace(get_mapped_vals(df=train_id, feature='id_30'), inplace=True)
train_id['id_31'].replace(get_mapped_vals(df=train_id, feature='id_31'), inplace=True)
train_id['id_33'].replace(get_mapped_vals(df=train_id, feature='id_33'), inplace=True)
train_id['DeviceInfo'].replace(get_mapped_vals(df=train_id, feature='DeviceInfo'), inplace=True)


train_id.head(10)
print(train_id.shape)

(144233, 48)


Preform Cleaning on Transactions

In [24]:
# Transormations:
true_false = {'T': 2, 'F': 1, None: 0}
train_txn['M1'].replace(true_false, inplace=True)
train_txn['M2'].replace(true_false, inplace=True)
train_txn['M3'].replace(true_false, inplace=True)
train_txn['M6'].replace(true_false, inplace=True)

# One Hot Encoding:
train_txn = one_hot_encoding(df=train_txn, column_name='ProductCD')
train_txn = one_hot_encoding(df=train_txn, column_name='card4')
train_txn = one_hot_encoding(df=train_txn, column_name='card6')
train_txn = one_hot_encoding(df=train_txn, column_name='P_emaildomain')

train_txn['M4'] = 'M4 ' + train_txn['M4'].astype(str)
train_txn = one_hot_encoding(df=train_txn, column_name='M4')

train_txn.head(10)
train_txn.shape

(590540, 291)

# XGBoost Model

### Combine the models together and train/test split


In [25]:
# I've chosen an outer join b/c we previously discusses that it is better to not reduce the rows to the size of train_id but to keep the rows from train_txn:
df = pd.merge(train_txn, train_id, on='TransactionID', how='inner')
df.shape

(144233, 338)

In [26]:
# Train/Test Split:
X = df.drop('isFraud', axis=1)
y = df['isFraud']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

### Create Model

In [28]:
# Create model:
xgb_clf = XGBClassifier()
xgb_clf.fit(train_x, train_y, eval_metric='aucpr')
# Print the accuracy score
print(xgb_clf.score(test_x, test_y))

0.9563559468922245


In [33]:
# Check if better then just guessing no every time:
not_fraud, fraud = df['isFraud'].value_counts()
fraud_propotion = fraud/not_fraud
print(f"The proportion of fraud is {fraud_propotion}.")
print(f"My model is {0.9565639407910701-(1-fraud_propotion)} percent better than guessing yes every time.")

The proportion of fraud is 0.08515216491742843.
My model is 0.041716105708498485 percent better than guessing yes every time.


### Get metrics on model

In [32]:
from sklearn.metrics import classification_report

predicted_proababilitis = xgb_clf.predict_proba(test_x)
precictions = xgb_clf.predict(test_x)
print(classification_report(test_y, precictions) )


              precision    recall  f1-score   support

           0       0.96      0.99      0.98     26596
           1       0.88      0.51      0.65      2251

    accuracy                           0.96     28847
   macro avg       0.92      0.75      0.81     28847
weighted avg       0.95      0.96      0.95     28847



This shows that the recall takes a big hit when trying to identify fraudelent transactions. I believe a wasy to better tackle this issue is to use SMOTE to tackle the class imbalance. Once the classes are balanced the model's recall and even precision should rise in fraudulent transactions.

### Use SMOTE to resolve class imbalance

> This will be attempted in next stage of modelling as this is only the initial model stage.



In [39]:
# from imblearn.over_sampling import SMOTE

In [40]:
# # Train/Test Split:
# X = df.drop('isFraud', axis=1)
# y = df['isFraud']

# # To be able to use SMOTE we must also impute all the values:
# # Th
# # Use SMOTE here
# oversample = SMOTE()
# X, y = oversample.fit_resample(X, y)

# train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)